In [3]:
from typing import List
import requests
import json
import pandas as pd
import numpy as np
import time
import sys
from collections.abc import MutableMapping,Iterable
import yaml


def get_API_response(url):
    response_API = requests.get(url)
    print('Status Code-', response_API.status_code)
    print('Loading Data--')
    start = time.time()
    raw_data = response_API.text
    # Parse Json
    data = json.loads(raw_data)
    print('Time taken to load data-',round((time.time()-start),3))
    return data
# Load api-keys
config_stream = open("../../config.yml",'r')
config = yaml.load(config_stream, Loader=yaml.BaseLoader)
etherscan_API_key = config['api_key']['etherscan']

In [6]:
api_key = etherscan_API_key

In [12]:
# Get Historical Ether Price
url_string = "https://api.etherscan.io/api?module=stats&action=ethdailyprice&startdate={0}&enddate={1}&sort={2}&apikey={3}" 
start_date = '2021-01-01'
end_date = '2021-01-01'
order = 'asc'
url = url_string.format(start_date, end_date, order, api_key)

In [15]:
data = get_API_response(url)
data

{'status': '0',
 'message': 'NOTOK',
 'result': 'Sorry, it looks like you are trying to access an API Pro endpoint. Contact us to upgrade to API Pro.'}

In [16]:
# Get total ether supply
url= f"https://api.etherscan.io/api?module=stats&action=ethsupply&apikey={api_key}"


In [18]:
data = get_API_response(url)
data

Status Code- 200
Loading Data--
Time taken to load data- 0.0


{'status': '1', 'message': 'OK', 'result': '121499119436500000000000000'}

In [27]:
#Returns the list of ERC-20 tokens transferred by an address, with optional filtering by token contract.
contract_address = '0x853d955aCEf822Db058eb8505911ED77F175b99e' # Frax
address = '0x4e83362442b8d1bec281594cea3050c8eb01311c'
url = f"https://api.etherscan.io/api?module=account&action=tokentx&contractaddress={contract_address}&sort=desc&apikey={api_key}"
# &startblock=0
# endblock=27025780&
# &page=1&offset=100
# &address={address}
data = get_API_response(url)

Status Code- 200
Loading Data--
Time taken to load data- 0.052


In [29]:
import datetime
datetime.datetime.fromtimestamp(1657724335)
# datetime.datetime(2020, 12, 16, 23, 12)

datetime.datetime(2022, 7, 13, 20, 28, 55)

In [36]:
len(data['result'])

10000

In [28]:
data['result'][0]
# 10000

{'blockNumber': '15134932',
 'timeStamp': '1657724335',
 'hash': '0xb5c3e1aff85d732c5509cd727fb661b1dd85ffb8bd6910019019ffdde010996d',
 'nonce': '4355',
 'blockHash': '0xdaed7438a2e0de31d3297ca7389db2a389b297ba8bf67f9e57f83ea0d8a3d2d6',
 'from': '0x9a834b70c07c81a9fcd6f22e842bf002fbffbe4d',
 'contractAddress': '0x853d955acef822db058eb8505911ed77f175b99e',
 'to': '0x8ce5796ef6b0c5918025bcf4f9ca908201b030b3',
 'value': '32186289704200578913044',
 'tokenName': 'Frax',
 'tokenSymbol': 'FRAX',
 'tokenDecimal': '18',
 'transactionIndex': '19',
 'gas': '415902',
 'gasPrice': '42555569079',
 'gasUsed': '342297',
 'cumulativeGasUsed': '3159894',
 'input': 'deprecated',
 'confirmations': '29'}

In [30]:
pdf_data = pd.json_normalize(data['result'], sep='_')
pdf_data['date'] = pd.to_datetime(pdf_data['timeStamp'] , unit='s')
pdf_data_test = pdf_data[['date','from','to','value','tokenName']]

In [31]:
pdf_data.tail(10)

,blockNumber,timeStamp,hash,nonce,blockHash,from,contractAddress,to,value,tokenName,tokenSymbol,tokenDecimal,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed,input,confirmations,date
9990,15037119,1656377283,0xd75dcdd44108d9ca8e1f3b844fdcbed589356d123b73...,208,0x4a921c7d26c7342eaa491c16de45eebc23aed2fbfce6...,0x67501e048c7fb5e59e73f52196d355bed670cf4f,0x853d955acef822db058eb8505911ed77f175b99e,0x8739f0eef3163c3db7b994d0e301bc375d757af6,12171378060476692163,Frax,FRAX,18,164,305841,47483422801,261241,15577222,deprecated,97842,2022-06-28 00:48:03
9991,15037119,1656377283,0xd75dcdd44108d9ca8e1f3b844fdcbed589356d123b73...,208,0x4a921c7d26c7342eaa491c16de45eebc23aed2fbfce6...,0x1e7b390149abb3b891e0a7027e111188032d4540,0x853d955acef822db058eb8505911ed77f175b99e,0x67501e048c7fb5e59e73f52196d355bed670cf4f,12171378060476692163,Frax,FRAX,18,164,305841,47483422801,261241,15577222,deprecated,97842,2022-06-28 00:48:03
9992,15037118,1656377266,0xf7dbd3405d680c8abe2ca22a7ae123db63afed79cf0f...,207,0x899175737afba8a92e2892b4316972031268093674d9...,0xfd0a40bc83c5fae4203dec7e5929b446b07d1c76,0x853d955acef822db058eb8505911ed77f175b99e,0x1e7b390149abb3b891e0a7027e111188032d4540,12171378060476692163,Frax,FRAX,18,294,180680,42375026456,125334,29522722,deprecated,97843,2022-06-28 00:47:46
9993,15037108,1656377082,0xb46a7b3e8763579eced2dafb99e7344adfbf0752ffd8...,303,0x3deba69c7575f721a4e32b0027a1c6eacdc3d47623e0...,0x9008d19f58aabd9ed0d60971565aa8510560ab41,0x853d955acef822db058eb8505911ed77f175b99e,0x6880f5334158980ecfded17ae18b455efce1c0b5,9198988657492860573328,Frax,FRAX,18,27,922732,46957425323,622980,4506908,deprecated,97853,2022-06-28 00:44:42
9994,15037108,1656377082,0xb46a7b3e8763579eced2dafb99e7344adfbf0752ffd8...,303,0x3deba69c7575f721a4e32b0027a1c6eacdc3d47623e0...,0x1111111254fb6c44bac0bed2854e76f90643097d,0x853d955acef822db058eb8505911ed77f175b99e,0x9008d19f58aabd9ed0d60971565aa8510560ab41,9198988657492860573328,Frax,FRAX,18,27,922732,46957425323,622980,4506908,deprecated,97853,2022-06-28 00:44:42
9995,15037108,1656377082,0xb46a7b3e8763579eced2dafb99e7344adfbf0752ffd8...,303,0x3deba69c7575f721a4e32b0027a1c6eacdc3d47623e0...,0x220bda5c8994804ac96ebe4df184d25e5c2196d4,0x853d955acef822db058eb8505911ed77f175b99e,0x1111111254fb6c44bac0bed2854e76f90643097d,9198988657492860573328,Frax,FRAX,18,27,922732,46957425323,622980,4506908,deprecated,97853,2022-06-28 00:44:42
9996,15037108,1656377082,0xb46a7b3e8763579eced2dafb99e7344adfbf0752ffd8...,303,0x3deba69c7575f721a4e32b0027a1c6eacdc3d47623e0...,0x220bda5c8994804ac96ebe4df184d25e5c2196d4,0x853d955acef822db058eb8505911ed77f175b99e,0xdd9f24efc84d93deef3c8745c837ab63e80abd27,9353656403942857640,Frax,FRAX,18,27,922732,46957425323,622980,4506908,deprecated,97853,2022-06-28 00:44:42
9997,15037108,1656377082,0xb46a7b3e8763579eced2dafb99e7344adfbf0752ffd8...,303,0x3deba69c7575f721a4e32b0027a1c6eacdc3d47623e0...,0xe1573b9d29e2183b1af0e743dc2754979a40d237,0x853d955acef822db058eb8505911ed77f175b99e,0x220bda5c8994804ac96ebe4df184d25e5c2196d4,9208342313896803430968,Frax,FRAX,18,27,922732,46957425323,622980,4506908,deprecated,97853,2022-06-28 00:44:42
9998,15037095,1656376885,0x3710889fdb5a9f210c913ef63216eea1a4ad80656731...,4265,0x0e543024c29da6d72c1f177ac503af412b61aed6a01e...,0x8300f0528e00ad33b218bb05d396f61a9fdd68cd,0x853d955acef822db058eb8505911ed77f175b99e,0xc2a856c3aff2110c1171b8f942256d40e980c726,7860994967536514350363,Frax,FRAX,18,7,353676,124202518216,293318,1782322,deprecated,97866,2022-06-28 00:41:25
9999,15037094,1656376835,0xbfa3e02f827ade042341fbfad6e28faf8e2e344cb180...,8563,0x6e93e16d2fe6fb2b129547adf351f62d03fd1621fd63...,0x8206412c107ef1adb70b9277974f5163760e128e,0x853d955acef822db058eb8505911ed77f175b99e,0x997b54d967e0452abf233eb12b6829db84f96c2a,18238991935369521996467,Frax,FRAX,18,49,600000,62106717438,275632,3604886,deprecated,97867,2022-06-28 00:40:35


In [32]:
pdf_data_test.tail()

,date,from,to,value,tokenName
9995,2022-06-28 00:44:42,0x220bda5c8994804ac96ebe4df184d25e5c2196d4,0x1111111254fb6c44bac0bed2854e76f90643097d,9198988657492860573328,Frax
9996,2022-06-28 00:44:42,0x220bda5c8994804ac96ebe4df184d25e5c2196d4,0xdd9f24efc84d93deef3c8745c837ab63e80abd27,9353656403942857640,Frax
9997,2022-06-28 00:44:42,0xe1573b9d29e2183b1af0e743dc2754979a40d237,0x220bda5c8994804ac96ebe4df184d25e5c2196d4,9208342313896803430968,Frax
9998,2022-06-28 00:41:25,0x8300f0528e00ad33b218bb05d396f61a9fdd68cd,0xc2a856c3aff2110c1171b8f942256d40e980c726,7860994967536514350363,Frax
9999,2022-06-28 00:40:35,0x8206412c107ef1adb70b9277974f5163760e128e,0x997b54d967e0452abf233eb12b6829db84f96c2a,18238991935369521996467,Frax


In [38]:
pdf_data_test.head(5)

,date,from,to,value,tokenName
0,2022-07-13 14:58:55,0x9a834b70c07c81a9fcd6f22e842bf002fbffbe4d,0x8ce5796ef6b0c5918025bcf4f9ca908201b030b3,32186289704200578913044,Frax
1,2022-07-13 14:58:26,0xf2f400c138f9fb900576263af0bc7fcde2b1b8a8,0xe1573b9d29e2183b1af0e743dc2754979a40d237,34927140613828096119498,Frax
2,2022-07-13 14:58:26,0xd632f22692fac7611d2aa1c0d552930d43caed3b,0xf2f400c138f9fb900576263af0bc7fcde2b1b8a8,34927140613828096119498,Frax
3,2022-07-13 14:57:21,0x97e7d56a0408570ba1a7852de36350f7713906ec,0xe1573b9d29e2183b1af0e743dc2754979a40d237,3728798070581678741545,Frax
4,2022-07-13 14:57:21,0x9a834b70c07c81a9fcd6f22e842bf002fbffbe4d,0x8300f0528e00ad33b218bb05d396f61a9fdd68cd,5332742849986033255663,Frax
